In [1]:
# the warnings in the libraries used can get old, do not show them
import warnings
warnings.filterwarnings('ignore')

In [2]:
from langchain_community.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [3]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline.from_model_id(model_id="flan-t5-large", task="text2text-generation", model_kwargs={"temperature":1e-10})

In [4]:
from langchain_community.vectorstores import FAISS

vectorestore = FAISS.load_local("ocp-llm-rag-vectorstore", hf)

In [5]:
#from langchain.document_loaders import TextLoader
#from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
#from langchain.llms.base import LLM
#from typing import Optional, List, Mapping, Any
from langchain import PromptTemplate, LLMChain
#from langchain.embeddings import HuggingFaceInstructEmbeddings 
#from langchain.agents import initialize_agent, Tool

template = "Cite one of the following pieces of context as an answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}Question: {question}\n\nAnswer:"
template = PromptTemplate(template=template, input_variables=["context", "question"])  
docsearcher = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", #stuff, map_reduce, refine, map_rerank
    chain_type_kwargs={"prompt": template},
    return_source_documents=True,
    retriever=vectorestore.as_retriever(search_type="similarity",search_kwargs={"k":1})) # similarity, mmr

In [6]:
def answer_question(question):
    # {query,result,source_documents{page_content,metadata{source,page}}}
    answer = docsearcher.invoke({"query": question})
    return {
        "query": answer['query'],
        "result": answer['result'],
        "metadata": answer['source_documents'][0].metadata,
        "page_content": answer['source_documents'][0].page_content,
    }

In [7]:
answer_question("what is openshift platform?")

Token indices sequence length is longer than the specified maximum sequence length for this model (809 > 512). Running this sequence through the model will result in indexing errors


{'query': 'what is openshift platform?',
 'result': 'OpenShift Container Platform is a cloud-based Kubernetes container platform',
 'metadata': {'source': 'https://access.redhat.com/documentation/en-us/openshift_container_platform/4.14/html-single/architecture'},
 'page_content': 'Red Hat OpenShift Documentation Team\n\nLegal Notice\n\nAbstract\n\nThis document provides an overview of the platform and application architecture in OpenShift Container Platform.\n\nChapter\xa01.\xa0Architecture overview\n\nOpenShift Container Platform is a cloud-based Kubernetes container platform. The foundation of OpenShift Container Platform is based on Kubernetes and therefore shares the same technology. To learn more about OpenShift Container Platform and Kubernetes, see product architecture.\n\n1.1.\xa0Glossary of common terms for OpenShift Container Platform architecture\n\nThis glossary defines common terms that are used in the architecture content.\n\naccess policies\n\nA set of roles that dictate

In [8]:
def answer_question_result(question):
    a = answer_question(question)
    text = a['page_content'].replace('\n', '<br>').replace('\t', '&#9;')
    return f"{a['result']} (<a href=\"{a['metadata']['source']}\" target=\"_blank\">source</a>)<br><hr>{text}"

In [9]:
answer_question_result("what is openshift platform?")

'OpenShift Container Platform is a cloud-based Kubernetes container platform (<a href="https://access.redhat.com/documentation/en-us/openshift_container_platform/4.14/html-single/architecture" target="_blank">source</a>)<br><hr>Red Hat OpenShift Documentation Team<br><br>Legal Notice<br><br>Abstract<br><br>This document provides an overview of the platform and application architecture in OpenShift Container Platform.<br><br>Chapter\xa01.\xa0Architecture overview<br><br>OpenShift Container Platform is a cloud-based Kubernetes container platform. The foundation of OpenShift Container Platform is based on Kubernetes and therefore shares the same technology. To learn more about OpenShift Container Platform and Kubernetes, see product architecture.<br><br>1.1.\xa0Glossary of common terms for OpenShift Container Platform architecture<br><br>This glossary defines common terms that are used in the architecture content.<br><br>access policies<br><br>A set of roles that dictate how users, applic

In [10]:
import gradio as gr

intf = gr.Interface(fn=answer_question_result, inputs="textbox", outputs="html")
intf.launch(inline=False)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
